In [1]:
import h5py
import numpy as np
from random import shuffle
from math import ceil
import matplotlib.pyplot as plt

In [2]:
hdf5_path = './dataset.hdf5'
batch_size = 50
nb_class = 2

File exists: True
File size: 800 bytes
